In [1]:
import pandas as pd

import spacy
from spacy import displacy

from spacy.matcher import Matcher
from spacy.matcher import DependencyMatcher
from spacy.tokens import Token

Token.set_extension("ignore", default=False, force=True)

nlp = spacy.load("de_dep_news_trf")

In [2]:
sentences = pd.read_csv('data/demo_data_real.csv')
sentence = sentences.iloc[1]['Original Text']
sentence

'Hallo Marcel, hier noch eine kurze Info für dich: Du kannst maximal 3 Urlaubstage mit ins nächste Jahr nehmen. Alle anderen Urlaubstage sollten bis Ende des Jahres verplant sein.'

# General Preprocessing

In [3]:
sentences.replace(to_replace=[r"\\t|\\n|\\r", "\t|\n|\r"], value=[" "," "], regex=True, inplace=True)

In [4]:
sentences

,ID,Bereinigter Text,Tags/Kategorie,Eingereicht von,Original Text
0,1,Die Tabelle “Subscriptions” im Schema “CRM” en...,"DWH, SQL",NaN,"Hey, du kannst die Tabelle “Subscriptions” im ..."
1,2,Maximal 3 Urlaubstage mit ins nächste Jahr neh...,HR,NaN,"Hallo Marcel, hier noch eine kurze Info für di..."
2,3,CR bedeutet Conversion Rate.,"Allgemeine Definitionen, CRM",NaN,"Hi Marcel, CR bedeutet in unserem Kontext Conv..."
3,4,AE bedeutet Account Executive.,"Allgemeine Definitionen, CRM",NaN,"AE = Account Executive, also Sales Mitarbeiter..."
4,5,Mit “Abrechnungsdatum” ist das Attribut “Purch...,"Allgemeine Definitionen, DWH",NaN,"Hi Marcel, ja genau mit “Abrechnungsdatum” ist..."
5,6,Die gebuchten Addons sind in der Tabelle “Subs...,"DWH, SQL",NaN,"Hey Marcel, die gebuchten Addons findest du in..."
6,7,"""Deactivated Date” steht für den Zeitpunkt, wa...","Allgemeine Definitionen, CRM",NaN,"Hi, das Attribut “Deactivated Date” steht für ..."
7,8,"Die wichtigsten Tabellen sind “Owners”, “Subsc...","CRM, DWH",NaN,"Hey Marcel, erstmal noch eine kurze Info für d..."
8,9,Mit dem Datenintegrationstool Stitch können wi...,DWH,NaN,"Hey Marcel, ich weiß nicht, ob du das Tool Sti..."
9,10,Als Werkstudent ist der Abrechnungszeitraum fü...,HR,NaN,"Hi Marcel, eine kurze Info noch für dich: Als ..."


# Generate Rules

In [5]:
doc = nlp(sentence)

In [6]:
# displacy.serve(doc, style='dep')

In [7]:
def set_ignore(matcher, doc, id, matches):
    for _, start, end in matches:
        for tok in doc[start:end]:
            tok._.ignore = True

## Grußformeln (Ende)

In [8]:
Grußformelmatcher = Matcher(nlp.vocab)
pattern = [
    [{"POS": "ADJ"}, {"POS": "NOUN"}, {"POS": "PROPN"}],
    [{"POS": "NOUN"}, {"POS": "PROPN"}],
    ]
Grußformelmatcher.add("Grußformeln", pattern, on_match=set_ignore)

In [9]:
matches = Grußformelmatcher(doc)

print(matches)

for match_id, start, end in matches:
    string_id = nlp.vocab.strings[match_id]  # Get string representation
    span = doc[start:end]  # The matched span
    print(match_id, string_id, start, end, span.text)

[]


## Begrüßungsformeln

In [10]:
Begrüßungsformelmatcher = Matcher(nlp.vocab)

pattern = [{"ORTH": "Hallo"}, {"POS": "PROPN"}, {"IS_PUNCT": True}]
pattern = [{"ORTH": "Hallo"}, {"POS": "ADV"}, {"IS_PUNCT": True}]

Begrüßungsformelmatcher.add("Begrüßungformeln", [pattern], on_match=set_ignore) # ,  

In [11]:
matches = Begrüßungsformelmatcher(doc)

print(matches)

for match_id, start, end in matches:
    string_id = nlp.vocab.strings[match_id]  # Get string representation
    span = doc[start:end]  # The matched span
    print(match_id, string_id, start, end, span.text)

[]


## Füllwortmatcher

In [12]:
Füllwortmatcher = Matcher(nlp.vocab)

pattern = [
    [{"lower": "erstmal"}],
    [{"lower": "erst"}, {"lower": "einmal"}],
    [{"lower": "ja"}, {"lower": "genau"}],
    [{"lower": "exakt"}],
    [{"lower": "jawohl"}],
    ]
Füllwortmatcher.add("Füllwörter", pattern, on_match=set_ignore)



In [13]:
matches = Füllwortmatcher(doc)

print(matches)

for match_id, start, end in matches:
    string_id = nlp.vocab.strings[match_id]  # Get string representation
    span = doc[start:end]  # The matched span
    print(match_id, string_id, start, end, span.text)

[]


## Resultat

In [14]:
toks = [tok.text + tok.whitespace_ for tok in doc if not tok._.ignore]
print("".join(toks))

Hallo Marcel, hier noch eine kurze Info für dich: Du kannst maximal 3 Urlaubstage mit ins nächste Jahr nehmen. Alle anderen Urlaubstage sollten bis Ende des Jahres verplant sein.


# Multiprocessing

In [15]:
# docs = nlp.pipe(sentences['Original Text'], n_process=4)

# Categorization

In [17]:
dataset = list(sentences[["Original Text", "Tags/Kategorie"]].sample(frac=1).itertuples(index=False, name=None))
train_data = dataset[:17]
dev_data = dataset[17:23]
test_data = dataset[23:27]
print(f"Total: {len(dataset)} - Train:  {len(train_data)} - Dev: {len(dev_data)} - Test: {len(test_data)}")

Total: 27 - Train:  17 - Dev: 6 - Test: 4


In [18]:
labels_test = sentences['Tags/Kategorie'].apply(lambda x : x.split(","))

all_labels = []
for li in labels_test:
    for label in li:
        label = label.strip()
        all_labels.append(label)

all_labels = set(all_labels)
print(all_labels)
all_labels = list(all_labels)
all_labels


{'Kubernetes', 'CRM', 'MLFlow', 'SQL', 'DWH', 'Zertifikate', 'Machine Learning', 'SAP', 'Allgemeine Definitionen', 'HR', 'DeepL', 'API', 'Python', 'ERP', 'Grafana'}


['Kubernetes',
 'CRM',
 'MLFlow',
 'SQL',
 'DWH',
 'Zertifikate',
 'Machine Learning',
 'SAP',
 'Allgemeine Definitionen',
 'HR',
 'DeepL',
 'API',
 'Python',
 'ERP',
 'Grafana']

In [19]:
doc = nlp(sentence)
for l in all_labels:
    print(l)
    if l in ['DWH', 'SQL']: 
        doc.cats[l] = 1
    else:
        doc.cats[l] = 0
print(doc.cats)
    

Kubernetes
CRM
MLFlow
SQL
DWH
Zertifikate
Machine Learning
SAP
Allgemeine Definitionen
HR
DeepL
API
Python
ERP
Grafana
{'Kubernetes': 0, 'CRM': 0, 'MLFlow': 0, 'SQL': 1, 'DWH': 1, 'Zertifikate': 0, 'Machine Learning': 0, 'SAP': 0, 'Allgemeine Definitionen': 0, 'HR': 0, 'DeepL': 0, 'API': 0, 'Python': 0, 'ERP': 0, 'Grafana': 0}


In [20]:
def convert(data, outfile):
    db = spacy.tokens.DocBin()
    docs = []
    for doc, label in nlp.pipe(data, as_tuples=True):
        for l in all_labels:
            if l in label: 
                doc.cats[l] = 1
            else:
                doc.cats[l] = 0
        db.add(doc)

        print(doc.cats)
    
    db.to_disk(outfile)

In [21]:
convert(train_data, "./train.spacy")
convert(dev_data, "./dev.spacy")
convert(test_data, "./test.spacy")

{'Kubernetes': 0, 'CRM': 1, 'MLFlow': 0, 'SQL': 0, 'DWH': 1, 'Zertifikate': 0, 'Machine Learning': 0, 'SAP': 0, 'Allgemeine Definitionen': 0, 'HR': 0, 'DeepL': 0, 'API': 0, 'Python': 0, 'ERP': 0, 'Grafana': 0}
{'Kubernetes': 0, 'CRM': 0, 'MLFlow': 0, 'SQL': 0, 'DWH': 0, 'Zertifikate': 0, 'Machine Learning': 0, 'SAP': 0, 'Allgemeine Definitionen': 0, 'HR': 0, 'DeepL': 1, 'API': 1, 'Python': 0, 'ERP': 0, 'Grafana': 0}
{'Kubernetes': 0, 'CRM': 0, 'MLFlow': 0, 'SQL': 0, 'DWH': 0, 'Zertifikate': 0, 'Machine Learning': 1, 'SAP': 0, 'Allgemeine Definitionen': 0, 'HR': 0, 'DeepL': 0, 'API': 0, 'Python': 1, 'ERP': 0, 'Grafana': 0}
{'Kubernetes': 0, 'CRM': 0, 'MLFlow': 0, 'SQL': 0, 'DWH': 1, 'Zertifikate': 0, 'Machine Learning': 0, 'SAP': 0, 'Allgemeine Definitionen': 0, 'HR': 0, 'DeepL': 0, 'API': 0, 'Python': 0, 'ERP': 1, 'Grafana': 0}
{'Kubernetes': 0, 'CRM': 0, 'MLFlow': 0, 'SQL': 0, 'DWH': 1, 'Zertifikate': 0, 'Machine Learning': 0, 'SAP': 0, 'Allgemeine Definitionen': 0, 'HR': 0, 'DeepL': 

In [22]:
!python -m spacy init config --lang pt --pipeline textcat_multilabel --optimize efficiency --force config.cfg

ℹ Generated config template specific for your use case
- Language: pt
- Pipeline: textcat_multilabel
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [25]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [26]:
!python -m spacy train config.cfg --paths.train ./train.spacy  --paths.dev ./dev.spacy --output model --verbose

[2022-11-28 14:38:54,113] [DEBUG] Config overrides from CLI: ['paths.train', 'paths.dev']
[2022-11-28 14:38:54,711] [INFO] Set up nlp object from config
[2022-11-28 14:38:54,721] [DEBUG] Loading corpus from path: dev.spacy
[2022-11-28 14:38:54,722] [DEBUG] Loading corpus from path: train.spacy
[2022-11-28 14:38:54,723] [INFO] Pipeline: ['textcat_multilabel']
[2022-11-28 14:38:54,728] [INFO] Created vocabulary
[2022-11-28 14:38:54,729] [INFO] Finished initializing nlp object
[2022-11-28 14:38:54,817] [INFO] Initialized pipeline components: ['textcat_multilabel']
[2022-11-28 14:38:54,829] [DEBUG] Loading corpus from path: dev.spacy
[2022-11-28 14:38:54,830] [DEBUG] Loading corpus from path: train.spacy


ℹ Saving to output directory: model
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['textcat_multilabel']
ℹ Initial learn rate: 0.001
E    #       LOSS TEXTC...  CATS_SCORE  SCORE 
---  ------  -------------  ----------  ------
  0       0           0.25       43.50    0.43
 21     200           7.71       39.00    0.39
 48     400           0.30       39.00    0.39
 82     600           0.10       39.00    0.39
124     800           0.05       40.33    0.40
175    1000           0.03       40.33    0.40
242    1200           0.02       40.33    0.40
317    1400           0.01       40.33    0.40
417    1600           0.01       40.33    0.40
✔ Saved pipeline to output directory
model\model-last
^C


In [27]:
!python -m spacy evaluate ./model/model-best/ ./test.spacy

ℹ Using CPU

================================== Results ==================================

TOK                   100.00
TEXTCAT (macro AUC)   16.11 
SPEED                 79973 


=========================== Textcat F (per label) ===========================

                             P      R      F
Kubernetes                0.00   0.00   0.00
CRM                       0.00   0.00   0.00
MLFlow                    0.00   0.00   0.00
SQL                       0.00   0.00   0.00
DWH                       0.00   0.00   0.00
Zertifikate               0.00   0.00   0.00
Machine Learning          0.00   0.00   0.00
SAP                       0.00   0.00   0.00
Allgemeine Definitionen   0.00   0.00   0.00
HR                        0.00   0.00   0.00
DeepL                     0.00   0.00   0.00
API                       0.00   0.00   0.00
Python                    0.00   0.00   0.00
ERP                       0.00   0.00   0.00
Grafana                   0.00   0.00   0.00


==================

ℹ Saving to output directory: model
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['textcat_multilabel']
ℹ Initial learn rate: 0.001
E    #       LOSS TEXTC...  CATS_SCORE  SCORE 
---  ------  -------------  ----------  ------
  0       0           0.25       43.50    0.43
 21     200           7.71       39.00    0.39
 48     400           0.30       39.00    0.39
 82     600           0.10       39.00    0.39
124     800           0.05       40.33    0.40
175    1000           0.03       40.33    0.40
242    1200           0.02       40.33    0.40
317    1400           0.01       40.33    0.40
417    1600           0.01       40.33    0.40
✔ Saved pipeline to output directory
model\model-last


[2022-11-28 14:41:48,357] [DEBUG] Config overrides from CLI: ['paths.train', 'paths.dev']
[2022-11-28 14:41:49,266] [INFO] Set up nlp object from config
[2022-11-28 14:41:49,281] [DEBUG] Loading corpus from path: dev.spacy
[2022-11-28 14:41:49,282] [DEBUG] Loading corpus from path: train.spacy
[2022-11-28 14:41:49,283] [INFO] Pipeline: ['textcat_multilabel']
[2022-11-28 14:41:49,293] [INFO] Created vocabulary
[2022-11-28 14:41:49,296] [INFO] Finished initializing nlp object
[2022-11-28 14:41:49,431] [INFO] Initialized pipeline components: ['textcat_multilabel']
[2022-11-28 14:41:49,451] [DEBUG] Loading corpus from path: dev.spacy
[2022-11-28 14:41:49,453] [DEBUG] Loading corpus from path: train.spacy
[2022-11-28 14:41:49,460] [DEBUG] Removed existing output directory: model\model-best
[2022-11-28 14:41:49,466] [DEBUG] Removed existing output directory: model\model-last


# Augmentation

In [33]:
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as nafc

from nlpaug.util import Action

In [57]:
text = 'Hallo Marcel, hier noch eine kurze Info für dich: Du kannst maximal 3 Urlaubstage mit ins nächste Jahr nehmen. Alle anderen Urlaubstage sollten bis Ende des Jahres verplant sein.'

In [38]:
aug = naw.SpellingAug()
augmented_texts = aug.augment(text, n=3)
print("Original:")
print(text)
print("Augmented Texts:")
print(augmented_texts)

Original:
Hallo Marcel, hier noch eine kurze Info für dich: Du kannst maximal 3 Urlaubstage mit ins nächste Jahr nehmen. Alle anderen Urlaubstage sollten bis Ende des Jahres verplant sein.
Augmented Texts:
['Halo marcel, hire noch eine kurze Info for dish: Do kannst maximal 2 Urlaubstage meet ins nächste Jahr nehmen. All anderen Urlaubstage sollten big Ende des Jahres verplant sein.', 'Hello Marcel, hire noch eine kurze Info for dish: Do kannst maximum three Urlaubstage meet ins nächste Jahr nehmen. All anderen Urlaubstage sollten big Ende des Jahres verplant sein.', 'Hallow Marcel, hair noch eine kurze Info for dish: Do kannst maximum 2 Urlaubstage my ins nächste Jahr nehmen. All anderen Urlaubstage sollten big Ende des Jahres verplant sein.']


In [41]:

import os
os.environ["MODEL_DIR"] = '../model'

In [42]:
aug = naw.ContextualWordEmbsAug(
    model_path='bert-base-uncased', action="insert")
augmented_text = aug.augment(text)
print("Original:")
print(text)
print("Augmented Text:")
print(augmented_text)

Original:
Hallo Marcel, hier noch eine kurze Info für dich: Du kannst maximal 3 Urlaubstage mit ins nächste Jahr nehmen. Alle anderen Urlaubstage sollten bis Ende des Jahres verplant sein.
Augmented Text:
['o hallo of marcel, hier noch eine von kurze info fur dich : ich du kannst maximal zu 3 2 urlaubstage mit ins nachste jahr nehmen. alle er anderen 5 urlaubstage wo sollten bis ende mit des jahres verplant sein.']


In [45]:
# Augmen by BERT
aug = naw.ContextualWordEmbsAug(model_path='bert-base-multilingual-uncased', aug_p=0.1)
augmented_text = aug.augment(text)
print("Original:")
print(text)
print("Augmented Text:")
print(augmented_text)

Original:
Hallo Marcel, hier noch eine kurze Info für dich: Du kannst maximal 3 Urlaubstage mit ins nächste Jahr nehmen. Alle anderen Urlaubstage sollten bis Ende des Jahres verplant sein.
Augmented Text:
['hallo marcel, hier noch das kurze info der dich : du kannst maximal einer urlaubstage mit ins nachste jahr gehen. seine anderen urlaubstage sollten bis ende des jahres verplant sein.']


In [51]:
aug = naw.SynonymAug(aug_src='ppdb', model_path='ppdb-1.0-l-lexical', aug_p=0.1)
augmented_text = aug.augment(text)
print("Original:")
print(text)
print("Augmented Text:")
print(augmented_text)

Original:
Hallo Marcel, hier noch eine kurze Info für dich: Du kannst maximal 3 Urlaubstage mit ins nächste Jahr nehmen. Alle anderen Urlaubstage sollten bis Ende des Jahres verplant sein.
Augmented Text:
['Hallo Marcel, hier noch eine kurz Info für dich: Du kannst maximale 3 Urlaubstage mit ins nächste Jahr übernehmen. Alle anderen Urlaubstage sollten solange Ende des Jahres verplant sein.']


In [52]:
aug = naw.RandomWordAug(action="swap")
augmented_text = aug.augment(text)
print("Original:")
print(text)
print("Augmented Text:")
print(augmented_text)

Original:
Hallo Marcel, hier noch eine kurze Info für dich: Du kannst maximal 3 Urlaubstage mit ins nächste Jahr nehmen. Alle anderen Urlaubstage sollten bis Ende des Jahres verplant sein.
Augmented Text:
['Marcel hallo, noch hier eine kurze Info dich für: Du maximal kannst Urlaubstage 3 mit ins Jahr nächste nehmen. Alle Urlaubstage anderen sollten bis Ende des Jahres sein verplant.']


In [53]:
aug = naw.RandomWordAug()
augmented_text = aug.augment(text)
print("Original:")
print(text)
print("Augmented Text:")
print(augmented_text)

Original:
Hallo Marcel, hier noch eine kurze Info für dich: Du kannst maximal 3 Urlaubstage mit ins nächste Jahr nehmen. Alle anderen Urlaubstage sollten bis Ende des Jahres verplant sein.
Augmented Text:
['Hallo Marcel, kurze Info dich: kannst maximal Urlaubstage mit ins Jahr nehmen. anderen Urlaubstage sollten bis Ende verplant sein.']


In [54]:
aug = naw.RandomWordAug(action='crop')
augmented_text = aug.augment(text)
print("Original:")
print(text)
print("Augmented Text:")
print(augmented_text)

Original:
Hallo Marcel, hier noch eine kurze Info für dich: Du kannst maximal 3 Urlaubstage mit ins nächste Jahr nehmen. Alle anderen Urlaubstage sollten bis Ende des Jahres verplant sein.
Augmented Text:
['Marcel Info für dich: Du kannst maximal 3 Urlaubstage mit ins nächste Jahr nehmen. Alle anderen Urlaubstage sollten bis Ende des Jahres']


In [59]:
import nlpaug.augmenter.word as naw
back_translation_aug = naw.BackTranslationAug(
    from_model_name='facebook/wmt19-de-en', 
    to_model_name='facebook/wmt19-en-de'
)
back_translation_aug.augment(text)

['Hallo Marcel, hier ist eine kurze Info für dich: Du kannst maximal 3 Tage Urlaub ins nächste Jahr nehmen, alle anderen Urlaube solltest du bis Ende des Jahres planen.']

In [ ]:
aug = nas.AbstSummAug(model_path='t5-base', num_beam=3)
augmented_text = aug.augment(article)
print("Original:")
print(article)
print("Augmented Text:")
print(augmented_text)